In [ ]:
install.packages("Rpdb")
install.packages("bio3d")
install.packages("htmltools")

In [ ]:
install.packages("remotes")
remotes::install_github("nvelden/NGLVieweR")

In [ ]:
library(httr)
library(xml2)

# Acceso a Pfam

- **Base de datos Pfam:**
  - Es una gran colección de familias de dominios de proteínas.
- **Cada familia Pfam:**
  - Representada por múltiples alineamientos de secuencias
    y un modelo oculto de Markov (HMM).
- **Entrada Pfam-A:**
  - Generalmente conocida como una entrada Pfam-A.
- **Contenido de cada entrada Pfam-A:**
  - Alineamiento de semillas curado que contiene un pequeño conjunto de miembros
    representativos de la familia.
  - Perfil HMM construido a partir del alineamiento de semillas.
  - Alineamiento completo generado automáticamente, que contiene todas las
    secuencias de proteínas detectables pertenecientes a la familia.
- **Generación del alineamiento completo:**
  - Realizado mediante búsquedas de HMM de perfil en bases de datos de
    secuencias primarias.
- **Se accede por la web de interpro.**
  - Previamente tenía una web propia.
- **URL**
  - Web: https://www.ebi.ac.uk/interpro/entry/pfam/#table
  - API: "https://www.ebi.ac.uk/interpro/api/"
  - API reference: https://pfam-docs.readthedocs.io/en/latest/api.html

## Recuperar datos de las familias disponibles

Uno de los servicios permite recuperar el nombre, código de acceso y descripción de todas las familias de proteínas disponibles.

In [ ]:
url <- "https://www.ebi.ac.uk/interpro/api/entry/integrated/pfam"
respuesta <- GET(
  url,
  query = list(
    type = "family",
    format = "json",
    page_size = 10 # 11320
  ),
)

In [ ]:
results <- content(respuesta, as = "parsed", encoding = "utf-8")
# results
names(results)

La respuesta tiene 4 campos:

- count: es el número total de resultados
- next: link a la siguiente página de resultados
- previous: links a la página de resultados anteriores
- results: tiene la lista de resultados:
  - cada resultado tiene un campo llamado metadata.
  - la metadata tiene:
    - el accession
    - el nombre
    - la base de datos de origen
    - el tipo de entrada
    - el identificador de Interpro

In [ ]:
result_df <- as.data.frame(
  t(
    sapply(
      results$results,
      function(x) return(unlist(x$metadata))
    )
  )
)
result_df

Podemos buscar por:

- repeat: motivo repetitivos
- domain: dominos de proteínas.
- coiled coil: motivos coiled-coil.
- disordered: regiones desordenadas.
- conserved site: sitios conservados.

## Recuperar anotaciones

In [ ]:
family_acc <- "PF00082"

url <- paste(
  "https://www.ebi.ac.uk/interpro/api/entry/pfam",
  family_acc,
  sep = "/"
)
respuesta <- GET(
  url,
  query = list(
    format = "json"
  ),
)

In [ ]:
content(respuesta)$metadata$name
content(respuesta)$metadata$description[[1]]$text
content(respuesta)$metadata$wikipedia$extract
content(respuesta)$metadata


## Buscar proteínas que contienen un dominio pfam

In [ ]:
pfam_domain <- "PF00082"

url <- paste(
  "https://www.ebi.ac.uk/interpro/api/protein/Uniprot/entry/pfam",
  pfam_domain,
  sep = "/"
)
url

In [ ]:
response <- GET(
  url,
  query = list(format = "json", page_size = 100),
)

In [ ]:
t(
  sapply(
    content(response, as = "parsed")$results,
    function(x) unlist(x$metadata)
  )
)

## Buscar dominios pfam en una proteína



In [ ]:

uniprot_acc <- "P0AA25" # Thioredoxin 1 de E. coli K 12

url <- paste(
  "https://www.ebi.ac.uk/interpro/api/entry/pfam/protein/Uniprot",
  uniprot_acc,
  sep = "/"
)
url

In [ ]:
response <- GET(
  url,
  query = list(format = "json", page_size = 100),
)

In [ ]:
t(
  sapply(
    content(response, as = "parsed")$results,
    function(x) unlist(x$metadata)
  )
)

## Buscar pfam domains en una secuencia

In [ ]:
## Buscar dominios en un secuencia de una proteína
url <- "https://www.ebi.ac.uk/Tools/services/rest/iprscan5/run"
seq <- "MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMNVQNGKWDSDPSGTKTCIDTKEGILQYCQEVYPELQITNVVEANQPVTIQNWCKRGRKQCKTHPHFVIPYRCLVGEFVSDALLVPDKCKFLHQERMDVCETHLHWHTVAKETCSEKSTNLHDYGMLLPCGIDKFRGVEFVCCPLAEESDNVDSADAEEDDSDVWWGGADTDYADGSEDKVVEVAEEEEVAEVEEEEADDDEDDEDGDEVEEEAEEPYEEATERTTSIATTTTTTTESVEEVVREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSAMSQSLLKTTQEPLARDPVKLPTTAASTPDAVDKYLETPGDENEHAHFQKAKERLEAKHRERMSQVMREWEEAERQAKNLPKADKKAVIQHFQEKVESLEQEAANERQQLVETHMARVEAMLNDRRRLALENYITALQAVPPRPRHVFNMLKKYVRAEQKDRQHTLKHFEHVRMVDPKKAAQIRSQVMTHLRVIYERMNQSLSLLYNVPAVAEEIQDEVDELLQKEQNYSDDVLANMISEPRISYGNDALMPSLTETKTTVELLPVNGEFSLDDLQPWHSFGADSVPANTENEVEPVDARPAADRGLTTRPGSGLTNIKTEEISEVKMDAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVVIATVIVITLVMLKKKQYTSIHHGVVEVDAAVTPEERHLSKMQQNGYENPTYKFFEQMQN"

body <- list(
  sequence = seq,
  email = "my_email@gmail.com",
  appl = "PfamA",
  goterms = "false",
  pathways = "false"
)

response <- POST(
  url,
  body = body
)

In [ ]:
job_id <- content(response)

In [ ]:
## Tengo que verificar el estado del trabajo
url <- paste(
  "https://www.ebi.ac.uk/Tools/services/rest/iprscan5/status",
  job_id,
  sep = "/"
)

response <- GET(url)

In [ ]:
content(response)

In [ ]:
## Recupero el resultado
url <- paste(
  "https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result",
  job_id,
  "json",
  sep = "/"
)

response <- GET(url)

In [ ]:
results <- content(response)

In [ ]:
t(
  sapply(
    results$results[[1]]$matches,
    function(x) {
      c(
        acceesion = x$signature$accession,
        name = x$signature$name,
        description = x$signature$description,
        type = x$signature$entry$type,
        start = x$locations[[1]]$start,
        end = x$locations[[1]]$end
      )
    }
  )
)


## Descargar alineamientos

Podemos descargas varios tipos de alineamientos de pfam


In [ ]:
pfam_accession <- "PF00085"

url <- paste(
  "https://www.ebi.ac.uk/interpro/wwwapi//entry/pfam",
  pfam_accession,
  sep = "/"
)

outfile <- "PF00085.sto.gz"

GET(
  url,
  query = list(
    annotation = "alignment:full",
    download = TRUE
  ),
  write_disk(outfile, overwrite = TRUE)
)


In [ ]:
library(R.utils)
gunzip(outfile)

Podemos descargar  proteomas representativos con diferente grado de indentidad de secuencia,
que pertenecen a una familia.

- RP15 Proteinas con 15% de identidad o mayor
- RP35 Proteinas con 35% de identidad o mayor
- RP55 Proteinas con 55% de identidad o mayor
- RP75 Proteinas con 75% de identidad o mayor

Para ello, hay que modificar la url.

# Acceso a RSCB PDB

Al igual que NCBI y Uniprot, PDB permite realizar búsqueda de texto.
Aunque estas son bastante más complicadas de realizar que en los servicios anteriores.

La consulta se tiene que escribir en formato **json**.

Vamos a provar con una búsquea simple.

La respuesta también está en formato json.

In [ ]:
library(httr)

In [ ]:
url <- "https://search.rcsb.org/rcsbsearch/v2/query"

query <- '{
  "request_options": {
    "return_all_hits": true
  },
  "query": {
    "type": "terminal",
    "service": "full_text",
    "parameters": {
      "value": "phosphofructokinase"
    }
  },
  "return_type": "entry"
}'

respuesta <- GET(url, query = list(json = query))

In [ ]:
pdb_search <- content(respuesta)

In [ ]:
t(sapply(pdb_search$result_set, unlist))

In [ ]:
# Recuperamos la cantidad de hits.
pdb_search$total_count

In [ ]:
pdb_ids <- sapply(pdb_search$result_set, function(x) x$identifier)

In [ ]:
library(bio3d)
library(Rpdb)

In [ ]:
# Podemos descargar el archivo PDB

bio3d::get.pdb(pdb_ids[1])

In [ ]:
# Podemos leer los datos del PDB
pdb1 <- bio3d::read.pdb('./1MTO.pdb')

In [ ]:
pdb1$atom

In [ ]:
library(NGLVieweR)

In [ ]:
a <- NGLVieweR("./1MTO.pdb") %>% addRepresentation("cartoon")